#### Load Libraries

In [1]:
import pandas as pd #load pandas library
import numpy as np #loads numpy library
import json #load JSON library
import math

import time #load TIME library
from datetime import datetime  #to convert data to datetime that does not fall within the pandas.to_datetime function timeframe

from shapely.geometry import Point  #transform latitude/longitude to geo-coordinate data

import matplotlib.pyplot as plt
%matplotlib inline

#### Load Config and Data Locations

In [2]:
#load config and csv file data with headers
datasrclocation = "datasets/MoCoTrafficViolations.csv"
datadstlocation = "tableau/MoCoTrafficViolations.csv"
configlocation = "config/config.json"

#### Load Data and peek into data

In [3]:
#load config and csv file data with headers
df = pd.read_csv(datasrclocation)
df.sample(1)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
1377908,1aa640ea-8612-4c38-ac00-669593e4c2dc,03/05/2013,12:32:00,MCP,Headquarters and Special Operations,EXCEEDING MAXIMUM SPEED: 70 MPH IN A POSTED 55...,N/B I-270 @ EXIT 5,39.071825,-77.161586,No,...,21-801.1,Transportation Article,False,WHITE,M,UPPER MARLBORO,MD,MD,B - Unmarked Patrol,"(39.0718254666667, -77.1615857666667)"


#### Data CleanUp

In [4]:
df['Violation Type'] = np.where(df['Violation Type'] == 'SERO', 'ESERO', df['Violation Type'])

In [5]:
df['SubAgency'] = np.where(df['SubAgency'] == 'S15', '3rd District, Silver Spring', df['SubAgency'])

In [6]:
df['SubAgency'] = np.where(df['SubAgency'] == 'W15', '4th District, Wheaton', df['SubAgency'])

In [7]:
df.sample(5)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
58557,b79e9602-515c-48d0-b4ea-15664f43effd,04/11/2019,20:42:00,MCP,"4th District, Wheaton",DRIVER FAIL TO MAKE REQUIRED STOP AT SIGN,GEORGIA AVE @ MONTPELIER RD,39.095523,-77.079555,No,...,21-202(m),Transportation Article,False,BLACK,M,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.0955233333333, -77.079555)"
793310,46e2014f-4183-4c0d-847e-b026c4dddafa,11/03/2015,09:31:00,MCP,"4th District, Wheaton",DRIVER FAILURE TO STOP AT STEADY CIRCULAR RED ...,GEORGIA AVE AT WINDHAM LN,39.031808,-77.048338,Yes,...,21-202(h1),Transportation Article,True,HISPANIC,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0318083333333, -77.0483383333333)"
1133648,1ccde1d5-1a61-4ede-b3f0-8d8367d22970,05/14/2014,18:45:00,MCP,"2nd District, Bethesda",OPERATOR NOT RESTRAINED BY SEATBELT,OLD GEORGETOWN RD @ ROCKVILLE PIKE,39.049642,-77.113543,No,...,22-412.3(b),Transportation Article,False,WHITE,M,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.0496416666667, -77.1135433333333)"
384093,811103ef-7b49-4ac4-b974-94cc8df5108e,10/03/2017,20:19:00,MCP,"2nd District, Bethesda",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,OLD GEORGETOWN RD / DEMOCRACY BLVD,39.024577,-77.126390,No,...,21-201(a1),Transportation Article,False,WHITE,M,POTOMAC,MD,MD,A - Marked Patrol,"(39.0245766666667, -77.12639)"
1417671,ce9fd9b7-0b84-424a-b1d2-8f0bcb5f8d4c,12/05/2012,00:39:00,MCP,"1st District, Rockville",EXCEEDING THE POSTED SPEED LIMIT OF 45 MPH,NB ROUTE 27 BETWEEEN OBSERVATION AND 355,39.211975,-77.248275,No,...,21-801.1,Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.21197475, -77.2482754833333)"


#### Define Maryland Transportation Code Functions

In [8]:
def MDTransCode(charge):
    switcher = {
            '1' : 'Title 1 - Definitions; General Provisions',
            '2' : 'Title 2 - Department of Transportation',
            '3' : 'Title 3 - Financing by Department',
            '4' : 'Title 4 - Revenue Facilities',
            '5' : 'Title 5 - Aviation',
            '6' : 'Title 6 - Ports',
            '7' : 'Title 7 - Mass Transit',
            '8' : 'Title 8 - Highways',
            '9' : 'Title 9 - Railroads',
            '10' : 'Title 10 - Transportation Compacts',
            '11' : 'Title 11 - Vehicle Laws -- Definitions; General Provisions',
            '12' : 'Title 12 - Vehicle Laws -- Motor Vehicle Administration',
            '13' : 'Title 13 - Vehicle Laws -- Certificates of  : Title and Registration of Vehicles',
            '14' : 'Title 14 - Vehicle Laws -- Antitheft Laws',
            '15' : 'Title 15 - Vehicle Laws -- Licensing of Businesses and Occupations',
            '16' : 'Title 16 - Vehicle Laws -- Drivers Licenses',
            '17' : 'Title 17 - Vehicle Laws -- Required Security',
            '18' : 'Title 18 - Vehicle Laws -- For-Rent Vehicles',
            '18.5' : 'Title 18.5. Peer-to-Peer Car Sharing Programs.',
            '19' : 'Title 19 - Vehicle Laws -- Civil Liability of Governmental Agencies',
            '20' : 'Title 20 - Vehicle Laws -- Accidents and Accident Reports',
            '21' : 'Title 21 - Vehicle Laws -- Rules of the Road',
            '22' : 'Title 22 - Vehicle Laws -- Equipment of Vehicles',
            '23' : 'Title 23 - Vehicle Laws -- Inspection of Used Vehicles and Warnings for Defective Equipment',
            '24' : 'Title 24 - Vehicle Laws -- Size, Weight, and Load; Highway Preservation',
            '25' : 'Title 25 - Vehicle Laws -- Respective Powers of State and Local Authorities; Disposition of Abandoned Vehicles',
            '26' : 'Title 26 - Vehicle Laws -- Parties and Procedure on Citation, Arrest, Trial, and Appeal',
            '27' : 'Title 27 - Vehicle Laws -- Penalties; Disposition of Fines and Forfeitures'
    }
    return switcher.get(charge,'Unknown')

In [9]:
def ESERO(charge):
    switcher = {
            '50' : 'Brakes',
            '51' : 'Tires',
            '52' : 'Steering',
            '53' : 'Exhaust System',
            '54*' : 'Windshield Wipers',
            '55*' : 'Headlights',
            '55' : 'Headlight aim',
            '56*' : 'Taillights',
            '57*' : 'Horn',
            '58' : 'Drivers Seat',
            '59' : 'Suspension / Shocks',
            '60*' : 'Bumpers',
            '61*' : 'Glass',
            '61' : 'TINT',
            '62*' : 'Load Cover',
            '64*' : 'Stop Lights',
            '65*' : 'Tag Light(s)',
            '66*' : 'Dash Lights',
            '67' : 'Wheel Alignment',
            '68*' : 'Rearview Mirrors',
            '69*' : 'Door Latch / Handle',
            '70' : 'Fuel System',
            '70*' : 'Cap',
            '71*' : 'Turn Signals',
            '72' : 'Wheels / Lugs',
            '73*' : 'Hood Catch(s)',
            '74' : 'Floor / Trunk Pan(s)',
            '76*' : 'Fender(s) / Flap(s)',
            '77' : 'Speedometer / Odometer',
            '78*' : 'Hazard Warning Lamp',
            '79*' : 'Park Lamp',
            '80*' : 'Side Marker Lamp',
            '81' : 'Fog / Auxiliary Driving Lamps',
            '82*' : 'Emergency Warning Lamp',
            '83*' : 'Back Up Lamp',
            '84*' : 'Reflectors',
            '85' : 'External Air Brake Component',
            '86*' : 'Low Air Warning Device'
    }
    return switcher.get(charge,"Unknown")

In [10]:
def MDTaxCode(charge):
    switcher = {
            '9' : 'Title 9 - Fuel Taxes'
    }
    return switcher.get(charge,"Unknown")

In [11]:
def MDBusiRegu(charge):
    switcher = {
            '10' : 'Title 10 - Motor Fuel and Lubricants'
    }
    return switcher.get(charge,"Unknown")

In [12]:
def getChargeTitleCat(article, charge):
    _chargeCharter = ''
    if (article=='Transportation Article') | (article=='Maryland Rules'):
        if ("-" in charge):
            chargeBaseCode = charge.split('-')[0]
            _chargeCharter = MDTransCode(chargeBaseCode)
            if _chargeCharter == 'Unknown':
                _chargeCharter = MDTransCode(charge)
        else:
            _chargeCharter = ESERO(chargeBaseCode)
            if _chargeCharter == 'Unknown':
                _chargeCharter = ESERO(charge)            
            
    elif (article=='BR'):
        if ("-" in charge):
            chargeBaseCode = charge.split('-')[0]
            _chargeCharter = MDBusiRegu(chargeBaseCode)
            if _chargeCharter == 'Unknown':
                _chargeCharter = MDBusiRegu(charge)            
            
    elif (article=='TG'):
        if ("-" in charge):
            chargeBaseCode = charge.split('-')[0]
            _chargeCharter = MDTaxCode(chargeBaseCode)
            if _chargeCharter == 'Unknown':
                _chargeCharter = MDTaxCode(charge)            
            
    elif (math.isnan(article)):
        _chargeCharter = ESERO(charge)
            
    return _chargeCharter

#### Add new empty column to dataset and add value Charge Charter value

In [13]:
df['Charge Charter'] =""

In [14]:
for il, row in df.iterrows():
    df.set_value(il,'Charge Charter',getChargeTitleCat(row['Article'],row['Charge']))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [15]:
df.iloc[457371]

SeqID                                   f2bcad2a-116c-4aac-a5b6-c2a7564340d6
Date Of Stop                                                      05/19/2017
Time Of Stop                                                        16:17:00
Agency                                                                   MCP
SubAgency                                              4th District, Wheaton
Description                DRIVER USING HANDS TO USE HANDHELD TELEPHONE W...
Location                                          RUSSELL AVE @ ODEN HAL AVE
Latitude                                                             39.1523
Longitude                                                           -77.1988
Accident                                                                  No
Belts                                                                     No
Personal Injury                                                           No
Property Damage                                                           No

#### Add additional new columns to assist with analysis

In [16]:
df['TrafViolYearI'] = pd.to_datetime(df['Date Of Stop']).dt.year

In [17]:
df['TrafViolMonthI'] = pd.to_datetime(df['Date Of Stop']).dt.month

In [18]:
df['TrafViolMonthDayI'] = pd.to_datetime(df['Date Of Stop']).dt.day

In [19]:
df['TrafViolHourI'] = pd.to_datetime(df['Time Of Stop']).dt.hour

In [20]:
df['TrafViolMonthN'] = pd.to_datetime(df['Date Of Stop']).dt.month_name()

In [21]:
df['TrafViolWeekdayN'] = pd.to_datetime(df['Date Of Stop']).dt.day_name()

In [22]:
df['TrafViolHour'] = pd.to_datetime(df['Time Of Stop']).dt.strftime('%I %p')

In [23]:
df.sample(5)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Arrest Type,Geolocation,Charge Charter,TrafViolYearI,TrafViolMonthI,TrafViolMonthDayI,TrafViolHourI,TrafViolMonthN,TrafViolWeekdayN,TrafViolHour
807491,6f8c8742-4bf8-4846-a2f5-898162df17c3,10/12/2015,10:49:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVING VEHICLE IN EXCESS OF REASONABLE AND PR...,IS 370 @ IS 270,39.120410,-77.201687,No,...,R - Unmarked Laser,"(39.12041, -77.2016866666667)",Title 21 - Vehicle Laws -- Rules of the Road,2015,10,12,10,October,Monday,10 AM
1133722,eb6fbaf9-1821-4e64-8445-a0763f2fde28,05/14/2014,16:26:00,MCP,"3rd District, Silver Spring",DRIVER FAILURE TO OBEY DESIGNATED LANE DIRECTIONS,NB 29 @ ICC (MD200),39.078040,-76.952062,No,...,A - Marked Patrol,"(39.07804, -76.9520616666667)",Title 21 - Vehicle Laws -- Rules of the Road,2014,5,14,16,May,Wednesday,04 PM
193889,1524cc1e-56a5-4d2c-b03f-e6ae6c77f013,08/24/2018,00:39:00,MCP,"2nd District, Bethesda",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,RIVER RD / BRAEBURN PKWY,0.000000,0.000000,No,...,A - Marked Patrol,"(0.0, 0.0)",Title 13 - Vehicle Laws -- Certificates of : ...,2018,8,24,0,August,Friday,12 AM
1282193,54c20af1-d16f-43ac-9022-1b6cf60251bb,09/05/2013,08:35:00,MCP,"3rd District, Silver Spring",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,COLESVILLE RD/ SOUTHWOOD AVE,39.026202,-77.011438,No,...,A - Marked Patrol,"(39.0262016666667, -77.0114383333333)",Title 21 - Vehicle Laws -- Rules of the Road,2013,9,5,8,September,Thursday,08 AM
1131130,7da3c7ff-2687-4660-bd3f-c368282b8be7,05/19/2014,11:48:00,MCP,"5th District, Germantown",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,RT. 117 @ LIBERTY MILL RD,39.166932,-77.277148,No,...,A - Marked Patrol,"(39.1669316666667, -77.2771483333333)",Title 13 - Vehicle Laws -- Certificates of : ...,2014,5,19,11,May,Monday,11 AM


In [24]:
df['AccidentI'] = np.where(df['Accident'] == 'Yes', 1, 0)

In [25]:
df['BeltsI'] = np.where(df['Belts'] == 'Yes', 1, 0)

In [26]:
df['PersonalInjuryI'] = np.where(df['Personal Injury'] == 'Yes', 1, 0)

In [27]:
df['PropertyDamageI'] = np.where(df['Property Damage'] == 'Yes', 1, 0)

In [28]:
df['FatalI'] = np.where(df['Fatal'] == 'Yes', 1, 0)

In [29]:
df['CommercialLicenseI'] = np.where(df['Commercial License'] == 'Yes', 1, 0)

In [30]:
df['HAZMATI'] = np.where(df['HAZMAT'] == 'Yes', 1, 0)

In [31]:
df['CommercialVehicleI'] = np.where(df['Commercial Vehicle'] == 'Yes', 1, 0)

In [32]:
df['AlcoholI'] = np.where(df['Alcohol'] == 'Yes', 1, 0)

In [33]:
df['WorkZoneI'] = np.where(df['Work Zone'] == 'Yes', 1, 0)

In [34]:
df.sample(5)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,AccidentI,BeltsI,PersonalInjuryI,PropertyDamageI,FatalI,CommercialLicenseI,HAZMATI,CommercialVehicleI,AlcoholI,WorkZoneI
492562,bf63b169-79d3-4307-93bb-a350220a11f3,03/14/2017,16:48:00,MCP,"4th District, Wheaton",DRIVER SPINNING WHEELS,2601 UNIVERSITY BLVD W,39.039492,-77.054655,No,...,0,0,0,0,0,0,0,0,0,0
396397,0fe2fe45-e364-4a74-a3a3-2c4473401c80,09/13/2017,00:28:00,MCP,"3rd District, Silver Spring",POSSESSING SUSPENDED LIC.,UNIVERSITY BLVD/E. WAYNE AVE,39.006760,-76.997490,No,...,0,0,0,0,0,0,0,0,0,0
1217727,07a3c560-2076-4614-ab6c-8bf6fe1c57f3,12/28/2013,12:58:00,MCP,"1st District, Rockville",Window Tint,HALPINE RD @ CHAPMAN AVE,39.061252,-77.121190,No,...,0,0,0,0,0,0,0,0,0,0
845332,050658ea-199a-41d6-93a7-ea1e7e25bf26,08/15/2015,08:23:00,MCP,"4th District, Wheaton",EXCEEDING THE POSTED SPEED LIMIT OF 30 MPH,GEORGIA @ GLENNALLEN,39.061123,-77.052765,No,...,0,0,0,0,0,0,0,0,0,0
543898,c07f8719-3864-4bc3-8f42-753c33915189,12/16/2016,23:51:00,MCP,"2nd District, Bethesda",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,KENILWORTH AVE AT STRATHMORE AVE,38.985805,-77.090462,No,...,0,0,0,0,0,0,0,0,0,0


In [35]:
dfMoCoTrafViol = df[(df['Latitude'] != 0.0) & (df['Longitude'] != 0.0)]

In [36]:
dfMoCoTrafViol.sample(5)

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,AccidentI,BeltsI,PersonalInjuryI,PropertyDamageI,FatalI,CommercialLicenseI,HAZMATI,CommercialVehicleI,AlcoholI,WorkZoneI
1394151,966137a6-ed95-46b1-b36c-3e520fbd6c20,01/29/2013,21:57:00,MCP,"3rd District, Silver Spring",FAILURE TO MAINTAIN LEGIBLE REGISTRATION PLATE...,STEWART LANE/ LOCKWOOD DR,39.043917,-76.980035,No,...,0,0,0,0,0,0,0,0,0,0
1339975,18c79cb1-d8b5-4807-be86-91f3464bea3a,05/19/2013,03:04:00,MCP,"2nd District, Bethesda",RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...,NORFOLK AVENUE/ FAIRMONT AVENUE,38.984104,-77.093080,No,...,0,0,0,0,0,0,0,0,0,0
211088,faa79ace-ba23-4e23-a7d7-0836d27fc1f2,07/24/2018,22:53:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVER FAILURE TO STOP AT STOP SIGN LINE,KENTLANDS BLVD/BEACON SQUARE CT,39.124542,-77.241401,No,...,0,0,0,0,0,0,0,0,0,0
988165,bb8cd2af-a18b-490f-ad50-dcedafa95aba,01/09/2015,18:52:00,MCP,"2nd District, Bethesda",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,OLD GEORGETOWN RD X 495,39.015430,-77.118842,No,...,0,0,0,0,0,0,0,0,0,0
971609,93fffe89-c96e-4810-a017-fc5d4bca4fa0,02/04/2015,13:31:00,MCP,"2nd District, Bethesda",DRIVING A MOTOR VEHICLE ON HIGHWAY WITHOUT REQ...,CONNECTICUT AVE/ I495,39.006142,-77.078872,No,...,0,0,0,0,0,0,0,0,0,0


In [37]:
dfMoCoTrafViol.to_csv(datadstlocation, index = None, header=True)

In [38]:
dfMoCoTrafViol.groupby(['SubAgency','Violation Type'])['SeqID'].count().rename("count")

SubAgency                                        Violation Type
1st District, Rockville                          Citation           68825
                                                 ESERO               6366
                                                 Warning            99200
2nd District, Bethesda                           Citation          109179
                                                 ESERO              11562
                                                 Warning           111945
3rd District, Silver Spring                      Citation          141856
                                                 ESERO              14133
                                                 Warning           130303
4th District, Wheaton                            Citation          159150
                                                 ESERO              17075
                                                 Warning           182853
5th District, Germantown                        

In [39]:
dfMoCoTrafViol.groupby(['SubAgency','TrafViolYearI','Violation Type'])['SeqID'].count().rename("count")

SubAgency                                        TrafViolYearI  Violation Type
1st District, Rockville                          2012           Citation           8266
                                                                ESERO               244
                                                                Warning           10792
                                                 2013           Citation           7403
                                                                ESERO               907
                                                                Warning           10114
                                                 2014           Citation          10830
                                                                ESERO              1308
                                                                Warning           12984
                                                 2015           Citation          10556
                                         

In [40]:
dfMoCoTrafViol.groupby(['SubAgency','TrafViolMonthN','Violation Type'])['SeqID'].count().rename("count")

SubAgency                            TrafViolMonthN  Violation Type
1st District, Rockville              April           Citation          6784
                                                     ESERO              670
                                                     Warning           9856
                                     August          Citation          5344
                                                     ESERO              571
                                                     Warning           8171
                                     December        Citation          4436
                                                     ESERO              448
                                                     Warning           6652
                                     February        Citation          5488
                                                     ESERO              503
                                                     Warning           8115
                    

In [41]:
dfMoCoTrafViol.groupby(['SubAgency','TrafViolWeekdayN','Violation Type'])['SeqID'].count().rename("count")

SubAgency                                        TrafViolWeekdayN  Violation Type
1st District, Rockville                          Friday            Citation          10500
                                                                   ESERO               908
                                                                   Warning           13475
                                                 Monday            Citation           9030
                                                                   ESERO               824
                                                                   Warning           12700
                                                 Saturday          Citation           6191
                                                                   ESERO               647
                                                                   Warning           10422
                                                 Sunday            Citation           6015
        

In [42]:
dfMoCoTrafViol.groupby(['SubAgency','TrafViolHour','Violation Type'])['SeqID'].count().rename("count")

SubAgency                            TrafViolHour  Violation Type
1st District, Rockville              01 AM         Citation          3296
                                                   ESERO              155
                                                   Warning           4607
                                     01 PM         Citation          2883
                                                   ESERO              227
                                                   Warning           4141
                                     02 AM         Citation          2952
                                                   ESERO               91
                                                   Warning           2457
                                     02 PM         Citation          3267
                                                   ESERO              255
                                                   Warning           3966
                                     03 AM    

In [43]:
dfMoCoTrafViol.groupby(['SubAgency','Gender','Violation Type'])['SeqID'].count().rename("count")

SubAgency                                        Gender  Violation Type
1st District, Rockville                          F       Citation           22773
                                                         ESERO               2086
                                                         Warning            37548
                                                 M       Citation           46031
                                                         ESERO               4280
                                                         Warning            61514
                                                 U       Citation              21
                                                         Warning              138
2nd District, Bethesda                           F       Citation           37831
                                                         ESERO               3877
                                                         Warning            45367
                          

#### Visualization dashboard and findings
https://public.tableau.com/profile/piyushpathak#!/vizhome/MoCoTrafViol/MCP